In [ ]:
import taichi as ti

ti.init(arch=ti.gpu)  # Run on GPU

quality = 1  
n_particles = 9000 * quality**2
n_grid = 128 * quality
dx = 1 / n_grid
inv_dx = float(n_grid)
dt = 1e-4 / quality



# Particle physical properties
p_vol = (dx * 0.5)**2
p_rho = 1
p_mass = p_vol * p_rho

# Material properties for jelly (red)
E, nu = 5e3, 0.2  # Young's modulus and Poisson's ratio
mu_0 = E / (2 * (1 + nu))
lambda_0 = E * nu / ((1 + nu) * (1 - 2 * nu))
drag_coefficient = 3  # Adjust this value to control drag strength


# Particle fields
x = ti.Vector.field(2, dtype=float, shape=n_particles)   # position
v = ti.Vector.field(2, dtype=float, shape=n_particles)     # velocity
C = ti.Matrix.field(2, 2, dtype=float, shape=n_particles)    # affine velocity field
F = ti.Matrix.field(2, 2, dtype=float, shape=n_particles)    # deformation gradient
material = ti.field(dtype=int, shape=n_particles)          # material id (we set to 0)
Jp = ti.field(dtype=float, shape=n_particles)              # plastic deformation (unused here)
object_id = ti.field(dtype=int, shape=n_particles)  # Assigns each particle to a specific object



# Grid fields
grid_v = ti.Vector.field(2, dtype=float, shape=(n_grid, n_grid))
grid_m = ti.field(dtype=float, shape=(n_grid, n_grid))

# External forces / interaction fields
gravity = ti.Vector.field(2, dtype=float, shape=())
attractor_strength = ti.field(dtype=float, shape=())
attractor_pos = ti.Vector.field(2, dtype=float, shape=())

@ti.kernel
def substep():
    # Reset grid nodes
    for i, j in grid_m:
        grid_v[i, j] = ti.Vector([0.0, 0.0])
        grid_m[i, j] = 0.0

    # Particle-to-Grid (P2G)
    for p in x:
        base = (x[p] * inv_dx - 0.5).cast(int)
        fx = x[p] * inv_dx - base.cast(float)
        # Quadratic interpolation weights
        w = [0.5 * (1.5 - fx)**2, 0.75 - (fx - 1)**2, 0.5 * (fx - 0.5)**2]
        
        # Update deformation gradient
        F[p] = (ti.Matrix.identity(float, 2) + dt * C[p]) @ F[p]
        
        # For jelly, we use a fixed hardening parameter.
        # Adjust h here: lower value -> softer material; higher value -> stiffer.
        h = 0.05
        mu = mu_0 * h
        la = lambda_0 * h
        
        # Compute SVD of the deformation gradient (for stress calculation)
        U, sig, V = ti.svd(F[p])
        J = 1.0
        for d in ti.static(range(2)):
            J *= sig[d, d]
        stress = 2 * mu * (F[p] - U @ V.transpose()) @ F[p].transpose() + \
                 ti.Matrix.identity(float, 2) * la * J * (J - 1)
        stress = (-dt * p_vol * 4 * inv_dx * inv_dx) * stress
        affine = stress + p_mass * C[p]
        
        # Scatter to grid nodes over a 3x3 neighborhood
        for i, j in ti.static(ti.ndrange(3, 3)):
            offset = ti.Vector([i, j])
            dpos = (offset.cast(float) - fx) * dx
            weight = w[i][0] * w[j][1]
            grid_v[base + offset] += weight * (p_mass * v[p] + affine @ dpos)
            grid_m[base + offset] += weight * p_mass

    # Grid update: convert momentum to velocity and add forces
    for i, j in grid_m:
        if grid_m[i, j] > 0:
            
            grid_v[i, j] = grid_v[i, j] / grid_m[i, j]  # Convert momentum to velocity
        
             # Apply gravity (which is zero for now)
            grid_v[i, j] += dt * gravity[None] * 30

            # Apply drag: reduce velocity based on drag coefficient
            grid_v[i, j] *= (1 - drag_coefficient * dt)  # Damping effect
            
           
            
            # Attractor force (from mouse interaction)
            dist = attractor_pos[None] - dx * ti.Vector([i, j])
            grid_v[i, j] += dist / (0.01 + dist.norm()) * attractor_strength[None] * dt * 100
            # Enforce simple boundary conditions
            if i < 3 and grid_v[i, j][0] < 0:
                grid_v[i, j][0] = 0
            if i > n_grid - 3 and grid_v[i, j][0] > 0:
                grid_v[i, j][0] = 0
            if j < 3 and grid_v[i, j][1] < 0:
                grid_v[i, j][1] = 0
            if j > n_grid - 3 and grid_v[i, j][1] > 0:
                grid_v[i, j][1] = 0

    # Grid-to-Particle (G2P)
    for p in x:
        base = (x[p] * inv_dx - 0.5).cast(int)
        fx = x[p] * inv_dx - base.cast(float)
        w = [0.5 * (1.5 - fx)**2, 0.75 - (fx - 1.0)**2, 0.5 * (fx - 0.5)**2]
        new_v = ti.Vector.zero(float, 2)
        new_C = ti.Matrix.zero(float, 2, 2)
        for i, j in ti.static(ti.ndrange(3, 3)):
            dpos = ti.Vector([i, j]).cast(float) - fx
            g_v = grid_v[base + ti.Vector([i, j])]
            weight = w[i][0] * w[j][1]
            new_v += weight * g_v
            new_C += 4 * inv_dx * weight * g_v.outer_product(dpos)
        v[p] = new_v
        C[p] = new_C
        x[p] += dt * v[p]

# Define the center positions as a Taichi field
center_positions = ti.Vector.field(2, dtype=float, shape=3)

@ti.kernel
def reset():
    radius = 0.1  # Radius of the circular objects
    group_size = n_particles // 3
    
    # Initialize the center positions **before** the loop
    center_positions[0] = [0.4, 0.4]
    center_positions[1] = [0.7, 0.4]
    center_positions[2] = [0.5, 0.7]

    for i in range(n_particles):
        group = i // group_size  # Determines which object the particle belongs to
        object_id[i] = group  # Store the object ID
        theta = ti.random() * 2 * ti.math.pi  # Random angle (0 to 2π)
        r = radius * ti.sqrt(ti.random())  # Ensures uniform distribution in the circle

        # Convert polar coordinates to Cartesian
        offset = ti.Vector([r * ti.cos(theta), r * ti.sin(theta)])
        x[i] = center_positions[group] + offset  # Now this works!

        material[i] = 1  # 1 is the jelly material
        v[i] = [0, 0]
        F[i] = ti.Matrix([[1, 0], [0, 1]])
        Jp[i] = 1
        C[i] = ti.Matrix.zero(float, 2, 2)


print("[Hint] Use WSAD/arrow keys to control gravity. Use left/right mouse buttons to attract/repel. Press R to reset.")
gui = ti.GUI("Taichi MLS-MPM - Jelly Only", res=512, background_color=0x112F41)
reset()
gravity[None] = [0, 0]

for frame in range(20000):
    if gui.get_event(ti.GUI.PRESS):
        if gui.event.key == "r":
            reset()
        elif gui.event.key in [ti.GUI.ESCAPE, ti.GUI.EXIT]:
            break
    if gui.event is not None:
        gravity[None] = [0, 0]
    if gui.is_pressed(ti.GUI.LEFT, "a"):
        gravity[None][0] = -1
    if gui.is_pressed(ti.GUI.RIGHT, "d"):
        gravity[None][0] = 1
    if gui.is_pressed(ti.GUI.UP, "w"):
        gravity[None][1] = 1
    if gui.is_pressed(ti.GUI.DOWN, "s"):
        gravity[None][1] = -1

    mouse = gui.get_cursor_pos()
    gui.circle((mouse[0], mouse[1]), color=0x336699, radius=15)
    attractor_pos[None] = [mouse[0], mouse[1]]
    attractor_strength[None] = 0
    if gui.is_pressed(ti.GUI.LMB):
        attractor_strength[None] = 1
    if gui.is_pressed(ti.GUI.RMB):
        attractor_strength[None] = -1

    for s in range(int(2e-3 // dt)):
        substep()
        
    # Render the particles in red; palette index 0 is used by every particle.
    gui.circles(x.to_numpy(), radius=1.5, palette=[0xED553B, 0xED553B], palette_indices=material)
    gui.show()


[Taichi] version 1.7.3, llvm 15.0.7, commit 5ec301be, osx, python 3.11.11


[I 03/21/25 15:24:23.281 52487927] [shell.py:_shell_pop_print@23] Graphical python shell detected, using wrapped sys.stdout


[Taichi] Starting on arch=metal
[Hint] Use WSAD/arrow keys to control gravity. Use left/right mouse buttons to attract/repel. Press R to reset.


In [5]:
material.to_numpy()

array([1, 1, 1, ..., 1, 1, 1], shape=(9000,), dtype=int32)